# Day 12 - Steering a ship

In [1]:
using System.IO;

var input = File.ReadAllLines(@".\input.txt")
    .Select(x => (instr: x[0], dist: Convert.ToInt32(x.Substring(1))))
    .ToArray();

input

index,Item1,Item2
0,F,35
1,L,90
2,S,5
3,F,4
4,R,90
5,F,46
6,W,3
7,N,1
8,L,90
9,F,13


Let's see if we can draw this madness

In [1]:
using Microsoft.DotNet.Interactive.Events;
int horiz = 80;
int vert = 10;
void FPS(DisplayedValue output, int faceDir, (int x, int y) curPos, (int x, int y) wp, int tick, int dist)
{
    var scene = $"tick: {tick}, x: {curPos.x}, y: {curPos.y}, dist: {dist}" + Environment.NewLine;
    for(var y = curPos.y - vert; y<curPos.y + vert; y++)
    {
        for(var x = curPos.x - horiz; x<curPos.x + horiz; x++)
        {
            if (x == curPos.x && y == curPos.y) {
                scene += faceDir switch {
                    0 => ">",
                    90 => "v",
                    180 => "<",
                    _ => "^"
                };
            } else if (x == wp.x && y == wp.y) {
                scene += "X";
            } else if (x % 21 == 0 && y % 11 == 0 || x % 27 == 0 && (y + 6) % 11 == 0) {
                scene += "~";
            } else {
                scene += " ";
            }
        }
        scene += Environment.NewLine;
    }

    output.Update(scene);
}


FPS(display(""), 90, (40, 7), (51, 7), 1, 1)

tick: 1, x: 40, y: 7, dist: 1
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                   ~                    ~                    ~                    ~                    ~                    ~                    ~              
                                                                                                                                                                
                                                                                                                                                                
    

In [1]:
using Microsoft.DotNet.Interactive.Events;
void Animate(DisplayedValue output, int faceDir, (int x, int y) from, (int x, int y) to, (int x, int y) wp, int tick, int speed = 1) {
    var diff = (x: to.x - from.x, y: to.y - from.y);
    var dist = Math.Max((int)Math.Sqrt(diff.x*diff.x+diff.y*diff.y), 1);
    var step = (float)speed/dist;
    var iter = (float)dist/speed;
    for(int i = 1, j = 1; i<iter; i+=speed, j++) {
        var inc = (x: step * j * diff.x, y: step * j * diff.y);
        var curr = (x: (int)((float)from.x + inc.x), y: (int)((float)from.y + inc.y));
        FPS(output, faceDir, curr, (x: curr.x + wp.x, y: curr.y + wp.y), tick, dist);
        System.Threading.Thread.Sleep(40);
    }
}

## 12.1 Solution

In [1]:
var prevPos = (x: 0, y: 0);
var curPos = (x: 0, y: 0);
var faceDir = 0;
var tick = 0;
var output = display(String.Join(Environment.NewLine, new String[21]));
FPS(output, faceDir, curPos, 1, 1);

foreach(var op in input)
{
    faceDir = op.instr switch {
        'L' => (faceDir - op.dist + 360) % 360,
        'R' => (faceDir + op.dist) % 360,
        _ => faceDir
    };

    prevPos = curPos;
    curPos = op.instr switch {
        'L' or 'R' => curPos,
        'F' => faceDir switch {
            0 => (x: curPos.x + op.dist, y: curPos.y), // east
            90 => (x: curPos.x, y: curPos.y + op.dist), // south
            180 => (x: curPos.x - op.dist, y: curPos.y), // west
            270 => (x: curPos.x, y: curPos.y - op.dist), // north
            _ => throw new Exception("Diagonals not supported")
        },
        'E' => (x: curPos.x + op.dist, y: curPos.y),
        'S' => (x: curPos.x, y: curPos.y + op.dist),
        'W' => (x: curPos.x - op.dist, y: curPos.y),
        'N' => (x: curPos.x, y: curPos.y - op.dist),
        _ => throw new Exception("Not supported")
    };

    Animate(output, faceDir, prevPos, curPos, curPos, tick++, 5);
    //System.Threading.Thread.Sleep(100);
    //FPS(output, faceDir, curPos);
}

Console.WriteLine(curPos.x + curPos.y);

tick: 756, x: 401, y: 189, dist: 8
                                                                                                                                                                
                                                                                                                                                                
   ~                          ~                          ~                          ~                          ~                          ~                     
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                

590


## 12.2 Sample solution

In [1]:
var sample = new[]{"F10", "N3", "F7", "L90", "F11"}
    .Select(x => (instr: x[0], dist: Convert.ToInt32(x.Substring(1))))
    .ToArray();

var itinerary = new List<(char op, int dist, int x, int y, int z, int i)>();
var curWP = (x: 10, y: -1);
var prevPos = (x: 0, y: 0);
var curPos = (x: 0, y: 0);
var faceDir = 0;
var tick = 0;

var output = display(String.Join(Environment.NewLine, new String[21]));
FPS(output, faceDir, curPos, 1, 1);

foreach(var op in sample)
{
    curWP = op.instr switch {
        'F' => curWP,
        'L' => (
            x: (int)Math.Round(curWP.x * Math.Cos(-1*op.dist/180.0*Math.PI) - curWP.y * Math.Sin(-1*op.dist/180.0*Math.PI)), 
            y: (int)Math.Round(curWP.x * Math.Sin(-1*op.dist/180.0*Math.PI) + curWP.y * Math.Cos(-1*op.dist/180.0*Math.PI))
        ),
        'R' => (
            x: (int)Math.Round(curWP.x * Math.Cos(op.dist/180.0*Math.PI) - curWP.y * Math.Sin(op.dist/180.0*Math.PI)), 
            y: (int)Math.Round(curWP.x * Math.Sin(op.dist/180.0*Math.PI) + curWP.y * Math.Cos(op.dist/180.0*Math.PI))
        ),
        'E' => (x: curWP.x + op.dist, y: curWP.y),
        'S' => (x: curWP.x, y: curWP.y + op.dist),
        'W' => (x: curWP.x - op.dist, y: curWP.y),
        'N' => (x: curWP.x, y: curWP.y - op.dist),
        _ => throw new Exception("Not supported")
    };

    faceDir = ((int)Math.Round(Math.Atan2(curWP.y, curWP.x)/(Math.PI*2)*4) * 90 + 360) % 360;

    prevPos = curPos;
    curPos = op.instr switch {
        'F' => (x: curPos.x + curWP.x * op.dist, y: curPos.y + curWP.y * op.dist),
        _ => curPos
    };

    tick++;
    if (op.instr == 'F') {
        Animate(output, faceDir, prevPos, curPos, curWP, tick, 2);
    }
    itinerary.Add((op.instr, op.dist, curWP.x, curWP.y, curPos.x, curPos.y));
}

Console.WriteLine(curPos.x + curPos.y);

itinerary

tick: 5, x: 148, y: -92, dist: 118
                                                                            X                                                                                   
                                                                                                                                                                
                                                                                                                                                                
                ~                    ~                    ~                    ~                    ~                    ~                    ~                 
                                                                                                                                                                
                                                                                                                                                                

-22


index,Item1,Item2,Item3,Item4,Item5,Item6
0,F,10,10,-1,100,-10
1,N,3,10,-4,100,-10
2,F,7,10,-4,170,-38
3,L,90,-4,-10,170,-38
4,F,11,-4,-10,126,-148


## 12.2 Solution

In [1]:
var itinerary = new List<(char op, int dist, int x, int y, int z, int i)>();
var curWP = (x: 10, y: -1);
var curPos = (x: 0, y: 0);
var prevPos = curPos;
var faceDir = 0;
var tick = 0;

var output = display(String.Join(Environment.NewLine, new String[21]));
FPS(output, faceDir, curPos, curPos, 1, 1);

foreach(var op in input)
{
    curWP = op.instr switch {
        'F' => curWP,
        'L' => (
            x: (int)Math.Round(curWP.x * Math.Cos(-1.0*op.dist/180.0*Math.PI) - curWP.y * Math.Sin(-1.0*op.dist/180.0*Math.PI)), 
            y: (int)Math.Round(curWP.x * Math.Sin(-1.0*op.dist/180.0*Math.PI) + curWP.y * Math.Cos(-1.0*op.dist/180.0*Math.PI))
        ),
        'R' => (
            x: (int)Math.Round(curWP.x * Math.Cos(op.dist/180.0*Math.PI) - curWP.y * Math.Sin(op.dist/180.0*Math.PI)), 
            y: (int)Math.Round(curWP.x * Math.Sin(op.dist/180.0*Math.PI) + curWP.y * Math.Cos(op.dist/180.0*Math.PI))
        ),
        'E' => (x: curWP.x + op.dist, y: curWP.y),
        'S' => (x: curWP.x, y: curWP.y + op.dist),
        'W' => (x: curWP.x - op.dist, y: curWP.y),
        'N' => (x: curWP.x, y: curWP.y - op.dist),
        _ => throw new Exception("Not supported")
    };

    prevPos = curPos;
    curPos = op.instr switch {
        'F' => (x: curPos.x + curWP.x * op.dist, y: curPos.y + curWP.y * op.dist),
        _ => curPos
    };

    faceDir = ((int)Math.Round(Math.Atan2(curWP.y, curWP.x)/(Math.PI*2)*4) * 90 + 360) % 360;

    tick++;
    if (op.instr == 'F') {
        Animate(output, faceDir, prevPos, curPos, curWP, tick, 5);
    }

    itinerary.Add((op.instr, op.dist, curWP.x, curWP.y, curPos.x, curPos.y));
}

Console.WriteLine(Math.Abs(curPos.x) + Math.Abs(curPos.y));

itinerary

Error: (40,9): error CS0103: The name 'Animate' does not exist in the current context